In [1]:
!pip install transformers==4.3.2

In [2]:
pip install torch==1.7

  Using cached torch-1.7.0-cp37-cp37m-win_amd64.whl (184.0 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: torch has an invalid wheel, .dist-info directory not found


In [3]:
!pip3 install --upgrade tensorflow-gpu --user

In [ ]:
!conda install -y -c conda-forge librosa

In [1]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import os
import pandas as pd

In [2]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h", )

In [3]:
tokens = {"_": 0, "<s>": 1, "</s>": 2, "<unk>": 3, "|": 4, "E": 5, "T": 6, "A": 7, "O": 8, "N": 9, "I": 10, "H": 11, "S": 12, "R": 13, "D": 14, "L": 15, "U": 16, "M": 17, "W": 18, "C": 19, "F": 20, "G": 21, "Y": 22, "P": 23, "B": 24, "V": 25, "K": 26, "'": 27, "X": 28, "J": 29, "Q": 30, "Z": 31}
rev_tokens = {value : key for (key, value) in tokens.items()}
extra_tokens = { 'bos_token' :"<s>" , 'eos_token' :"</s>" , 'unk_token' : "<unk>" , 'pad_token' : "_" , 'word_delimiter_token' : "|" }
rev_extra_tokens = {value : key for (key, value) in extra_tokens.items()}

In [4]:
def audio2ctc(audio):
    audio, rate = librosa.load(audio, sr = 16000, )
    input_values = tokenizer(audio, return_tensors = "pt").input_values
    logits = model(input_values).logits
    prediction = torch.argmax(logits, dim = -1)
    transcription = tokenizer.batch_decode(prediction)[0]
    ctc = ''
    for a in prediction.numpy()[0]:
        ctc += rev_tokens[a]
    return ctc,transcription

In [15]:
max_len = 0
ids = []
ctc_texts = []
trans = []
for speaker in os.listdir('grid_corpus/audio/'):
    print(speaker)
    if speaker == 's21':
        continue
    for file in os.listdir('grid_corpus/audio/'+speaker+'/'):
        file_path = 'grid_corpus/audio/'+speaker+'/'+file
        ctc_text ,transcription = audio2ctc(file_path)
        ctc_texts.append(ctc_text)
        trans.append(transcription)
        ids.append(speaker+'_'+file[:-4])
        max_len = max(max_len,len(ctc_text))
ctc_textss = []
for text in ctc_texts:
    ctc_textss.append(text+''.join(['_' for i in range(max_len - len(text))]))

s1
s10
s11
s12
s13
s14
s15
s16
s17
s18
s19
s2
s20
s21
s22
s23
s24
s25
s26
s27
s28
s29
s3
s30
s31
s32
s33
s34
s4
s5
s6
s7
s8
s9


In [16]:
df = pd.DataFrame(columns = ['ids','ctc_text','transcriptions'])
df['ids'] = ids
df['ctc_text'] = ctc_textss
df['transcriptions'] = trans

In [17]:
df.to_csv('y_label.csv')